In [ ]:
conda install -n base ipykernel --update-deps --force-reinstall

In [ ]:
import pandas as pd
import nltk
import re
import numpy as np
nltk.download('punkt')

: 

In [ ]:
filename = 'articles.pkl'

article_df = pd.read_pickle(filename)
article_df = article_df.assign(Article_Number=range(len(article_df)))
article_df = article_df.reset_index()
article_df.info()

In [ ]:
# tokenize sentences in an article 
def split_sentences(article, article_id):
    sentences = nltk.sent_tokenize(article)
    sentences_with_id = [(sentence, article_id) for sentence in sentences]
    return sentences_with_id

sentences_list = []

# add sentences to a new DF along with article ID 
for article, article_id in article_df[['content','Article_Number']].values:
    sentences = split_sentences(str(article), article_id)
    sentences_list.extend(sentences)

sentences_df = pd.DataFrame(sentences_list, columns= ['sentences', 'article_id'])

# TO-DO: add the data & find the pronouns to determine gender

In [ ]:
sentences_df

In [ ]:
#sentences_df.drop(['female_count', 'male_count'], axis = 1)

In [ ]:
his_w = [' his ', ' he ']
his_w_expand = ["his", "he", "man", "uncle", "dad", "daddy", "father", "boy", "husband"]
her_w = [" her ", " she "]
her_w_expand = ["her", "she", "woman", "aunt", "mother","mom", "mommy", "girl", "wife"]


def pronoun_occurance(text, female_list, male_list):
    female_count = sum(text.count(word) for word in female_list)
    male_count = sum(text.count(word) for word in male_list)
    return female_count, male_count

#sentences_df['female_count'], sentences_df['male_count'] = zip(sentences_df['sentences'].apply(pronoun_occurance, female_list=her_w, male_list=his_w))


In [ ]:
#this solution fixes the bug of double counting 
def pronoun_occurances(text):
    """ This function will count the number of female and male pronoun occurences in a given sentence. 
    We will need to update the REGEX in order to incorporate more pronouns if we need to"""
    pattern_m = r'(\s|^)(he|his)\b' #this regex will capture he/his as standalone words within a string but also at beginning of sentence
    matches_m = re.findall(pattern_m, text, re.IGNORECASE) #IGNORECASE is necessary to make sure that it picks up the pronouns at the beginning of a sentence
    pattern_f = r'(\s|^)(she|her)\b'
    matches_f = re.findall(pattern_f, text, re.IGNORECASE)
    count_m = len(matches_m)
    count_f = len(matches_f)
    return count_f, count_m

In [ ]:
#create a variable applying the function of pronoun occurence
sent = sentences_df['sentences'].apply(pronoun_occurances)
# Create two new columns in sentences DF from the tuple output in "sent"
sentences_df['female_count'] = [x[0] for x in sent]
sentences_df['male_count']= [x[1] for x in sent]

#Bug is fixed and now it counts properly
sentences_df

In [ ]:
sentences_df.loc[[7]] #works

In [ ]:
def compare_count(male_col, female_col): 
    """This function compares the count of female to male pronouns. It will output "1" if male count bigger
    than female count, "neutral" if the count is equal, and "female" if there is a higher female count. 
    The function returns strings because we need categorical variables for log reg to run"""
    if male_col > female_col: 
        return "1"
    elif male_col == female_col: 
        return "0.5"
    else:
        return "0"

sentences_df['col_type'] = sentences_df.apply(lambda row: compare_count(row['male_count'], row['female_count']), axis=1)
sentences_df



**Building the Classifier**

In [ ]:
sentences_df.head(25)

**Logistics Regression Classifier**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 

In [ ]:
#Sentence Encoding

sentences_df['encoded_sentences'] = sentences_df.loc[:,'sentences']

def tidy_text(sentence):
   
    # Convert words to lower case
    sentence = sentence.lower()
    
    # Expand contractions
    if True:
        sentence = sentence.split()
        new_text = []
        for word in sentence:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        sentence = " ".join(new_text)
    
    # Format words and remove unwanted characters
    sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence, flags=re.MULTILINE)
    sentence = re.sub(r'\<a href', ' ', sentence)
    sentence = re.sub(r'&amp;', '', sentence) 
    sentence = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', sentence)
    sentence = re.sub(r'<br />', ' ', sentence)
    sentence = re.sub(r'\'', ' ', sentence)
    
    # remove stopwords
    if remove_stopwords:
        sentence = sentence.split()
        stops = set(stopwords.words("english"))
        sentence = [w for w in sentence if not w in stops]
        sentence = " ".join(sentence)

    # Tokenize each word
    sentence =  nltk.WordPunctTokenizer().tokenize(sentence)
    
    # Lemmatize each token
    lemm = nltk.stem.WordNetLemmatizer()
    sentence = list(map(lambda word:list(map(lemm.lemmatize, word)), sentence))
    
    return sentence

sentence_df['encoded_sentences'] = sentence_df['encoded_sentences'].apply(tidy_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_converter = CountVectorizer(tokenizer=lambda doc: doc)

#this should be changed to only the male and female sents. 
x = bow_converter.fit_transform(sentence_df['encoded_sentences'])
words = bow_converter.get_feature_names()
len(words)

In [ ]:
#splitting the data into training and test set 
X = sentences_df[["female_count", "male_count"]] #input 
y = sentences_df["col_type"] #label 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

#Note: using sentences_df["sentences"] creates an error 

# I think we are getting an error here because we haven't encoded the sentences. I'm not sure if I should work on this or the data set. I'm thinking BOW 
#and TFIDF for what we should use to encode. Also I do think we need to clean up the sentences. ie punctuation, lowercase, contractions, tokenazion, 
# and maybe stemming? 


In [ ]:
#multinomial logistic regression 
logreg = LogisticRegression(multi_class = "multinomial", solver = "lbfgs") #classifier 
logreg.fit(X_train, y_train) #fit the model 
logreg.score(X_train, y_train) #get accuracy

In [ ]:
logreg.coef_

In [ ]:
pd.DataFrame(zip(X_train.columns, np.transpose(abs(logreg.coef_))), columns=['features', 'coef']) #use absolute values to identify biggest coeffs

Interestingly, male coefficient for neutral is higher than the female. Otherwise, as expected, the coefficients for female and male each are correspondingly high for each gender. 

In [ ]:
prediction = logreg.predict(X_test) #predict test 
print(metrics.accuracy_score(y_test, prediction)) #accuracy 
print(metrics.confusion_matrix(y_test, prediction)) #confusion matrix
print(metrics.classification_report(y_test, prediction))

In [ ]:
#I think with the above solution we wouldn't need this function 
def count_words(text, word_list):
    return sum(text.count(word) for word in word_list)

sentences_df['male_count2'] = sentences_df['sentences'].apply(count_words, word_list=his_w)
sentences_df['female_count2'] = sentences_df['sentences'].apply(count_words, word_list=her_w)


In [ ]:
pd.set_option('display.max_colwidth', 1000)
sentences_df.loc[[7]]

#there is a bug here. It weirdly seems to be double counting? The zip function is new to me though so maybe thats the problem. 

In [ ]:
df = pd.DataFrame({'text': ['This is a sample text', 'Another text example', 'One more example']})

# define two lists of specific words to count
word_list1 = ['text', 'example']
word_list2 = ['is', 'more']

def count_words(text, word_list):
    return sum(text.count(word) for word in word_list)

# use apply() to add two new columns with the counts of the specific words in each list
df['word_count1'] = df['text'].apply(count_words, word_list=word_list1)
df['word_count2'] = df['text'].apply(count_words, word_list=word_list2)

# print the resulting DataFrame
print(df)

